In [1]:
import cv2
import numpy as np
import math
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import random
from mpl_toolkits.mplot3d import Axes3D

In [2]:
color_selection_path = r"\\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT"
images_to_process = r"\\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_S_IHC"
processed_images_path = r"\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\ben_processed_Synthetic_static"
downscale_factor = 2 #only for tile creation
tile_size = 200
mask_output_type = ".png"
file_type = ".png"

In [3]:

# Global variables
drawing = False
ix, iy = -1, -1
mask = None  # Initialize mask
save_rectangle = False

zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
mouseRGB_last_x = 0
mouseRGB_last_y = 0
coordinates = []

def mouseRGB(event, x, y, flags, param):
    global zoom_factor, pan_x, pan_y, cursor_x, cursor_y, drawing, mouseRGB_last_x, mouseRGB_last_y, display_img, img, downscale_factor, coordinates

    if event == cv2.EVENT_LBUTTONDOWN:
        mouseRGB_last_x, mouseRGB_last_y = x, y

    elif event == cv2.EVENT_RBUTTONDOWN:
        edge_len = tile_size * downscale_factor
        click_center_x, click_center_y = x, y

        M = np.float32([[zoom_factor, 0, cursor_x - cursor_x * zoom_factor + pan_x], [0, zoom_factor, cursor_y - cursor_y * zoom_factor + pan_y]])
        inv_M = cv2.invertAffineTransform(M)
        orig_center = cv2.transform(np.array([[[click_center_x, click_center_y]]]), inv_M)[0][0]

        orig_top_left = (int(orig_center[0] - edge_len // 2), int(orig_center[1] - edge_len // 2))
        orig_bottom_right = (int(orig_center[0] + edge_len // 2), int(orig_center[1] + edge_len // 2))

        cv2.rectangle(img,
                      orig_top_left,
                      orig_bottom_right,
                      (0, 0, 255), 2)
        coordinates.append([orig_top_left[0], orig_top_left[1], orig_bottom_right[0], orig_bottom_right[1]])
        update_display_img()

    elif event == cv2.EVENT_MOUSEWHEEL:
        cursor_x, cursor_y = x, y
        zoom_factor += 0.1 if flags > 0 else -0.1
        zoom_factor = max(0.1, zoom_factor)
        update_display_img()

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            pan_x += x - mouseRGB_last_x
            pan_y += y - mouseRGB_last_y
            mouseRGB_last_x, mouseRGB_last_y = x, y
            update_display_img()

def update_display_img():
    global display_img, zoom_factor, pan_x, pan_y, cursor_x, cursor_y, img
    h, w = img.shape[:2]
    center_x, center_y = cursor_x, cursor_y
    M = np.float32([[zoom_factor, 0, center_x - center_x * zoom_factor + pan_x],
                    [0, zoom_factor, center_y - center_y * zoom_factor + pan_y]])
    display_img = cv2.warpAffine(img, M, (w, h))
    cv2.imshow('image', display_img)

# Load images from the directory
mypath = color_selection_path
onlyfiles = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
# Randomly select 9 files
if len(onlyfiles) > 4:
    selected_files = random.sample(onlyfiles, 4)
else:
    selected_files = onlyfiles
images = []

for read_path in selected_files:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img_full = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img_full is None:
        print(f"Failed to load image at {final_path}")
        continue

    # Initialize/reset global variables
    drawing = False
    ix, iy = -1, -1
    mask = None  # Initialize mask
    save_rectangle = False

    zoom_factor = 1.0
    pan_x, pan_y = 0, 0
    cursor_x, cursor_y = 0, 0
    mouseRGB_last_x = 0
    mouseRGB_last_y = 0

    height, width = img_full.shape[:2]
    print(height, width)
    img = cv2.resize(img_full, (int(width * downscale_factor), int(height * downscale_factor)), interpolation=cv2.INTER_LINEAR)
    display_img = None
    coordinates = []

    cv2.imshow('image', img)
    cv2.setMouseCallback('image', mouseRGB)

    # Wait for ESC key to exit or 'x' to save rectangles
    while True:
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # ESC key
            break
        elif key == ord('x'):
            save_rectangle = True  # Change color to blue

    # Process rectangles and save cropped images
    for coord in coordinates:
        print(f"Cropping coordinates: {coord}")
        top_left = (int(coord[0] / downscale_factor), int(coord[1] / downscale_factor))
        bottom_right = (int(coord[2] / downscale_factor), int(coord[3] / downscale_factor))
        cropped_image = img_full[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        if cropped_image.size == 0:
            print(f"Warning: Cropped image is empty for coordinates: {coord}")
            continue
        print(cropped_image.shape)
        images.append(cropped_image)

    cv2.destroyAllWindows()


Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\167_tile_00207.png
256 256
Cropping coordinates: [33, 49, 433, 449]
(200, 200, 3)
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\192_tile_00192.png
256 256
Cropping coordinates: [33, 49, 433, 449]
(200, 200, 3)
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\192_tile_00143.png
256 256
Cropping coordinates: [81, 48, 481, 448]
(200, 200, 3)
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\102_tile_00190.png
256 256
Cropping coordinates: [28, 63, 428, 463]
(200, 200, 3)


In [4]:
def concatenate_images(images, rows, cols):
    """
    Concatenate multiple images of the same size into a grid.
    :param images: List of images to concatenate
    :param rows: Number of rows in the grid
    :param cols: Number of columns in the grid
    :return: Concatenated image
    """
    assert len(images) <= rows * cols, "Number of images is greater than grid size"

    # Ensure all images are the same size
    assert all(img.shape == images[0].shape for img in images), "All images must have the same dimensions"

    # Create rows of images
    rows_of_images = [cv2.hconcat(images[i:i+cols]) for i in range(0, len(images), cols)]

    # Concatenate rows vertically
    return cv2.vconcat(rows_of_images)

num_imgs = len(images)
x_grid = int(math.sqrt(num_imgs))
if num_imgs % x_grid == 0:
    y_grid = x_grid
else:
    y_grid = x_grid + 1

if images:
    result = concatenate_images(images, y_grid, x_grid)
else:
    print("No images to concatenate.")

In [5]:
# Initialize lists and index
stain_colors = ["brown", "blue", "red"]
hsv_values = [[] for _ in stain_colors]  # List to store HSV values for each color
hsv_index = 0

def set_hsv_index(value):
    global hsv_index
    try:
        hsv_index = int(value)
        if 0 <= hsv_index < len(stain_colors):
            print(f"HSV index set to: {hsv_index} ({stain_colors[hsv_index]})")
        else:
            print(f"Invalid index. Please enter a number between 0 and {len(stain_colors) - 1}.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

# Function to display the HSV values on mouse click
def get_hsv_values(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN or event == cv2.EVENT_RBUTTONDOWN:
        hsv_value = hsv_image[y, x]
        print(f"Mouse Event: {event}, HSV Value at ({x}, {y}): {hsv_value}")
        if event == cv2.EVENT_RBUTTONDOWN:
            if 0 <= hsv_index < len(stain_colors):
                hsv_values[hsv_index].append(hsv_value)
                print(f"HSV Value added to index {hsv_index}: {hsv_value}")
            else:
                print("Invalid HSV index. Cannot add value.")

# Convert the image to HSV color space
hsv_image = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)

# Create a resizable window and set a mouse callback
cv2.namedWindow('Image', cv2.WINDOW_KEEPRATIO)
cv2.setMouseCallback('Image', get_hsv_values)
input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
set_hsv_index(input_value)

while True:
    cv2.imshow('Image', result)
    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # ESC key to break
        break
    elif key == ord('c'):  # Press 'c' key to change HSV index
        # Ask the user to input the HSV index
        input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
        set_hsv_index(input_value)

cv2.destroyAllWindows()

# Initialize lists to store max and min values for each color category
max_values_list = []
min_values_list = []

# Calculate max and min values separately for each color category
for i in hsv_values:  # Iterating over each color category
    if i:  # Ensure there are values to compute max and min
        max_values = np.amax(i, axis=0).tolist()  # Convert to list for better readability
        min_values = np.amin(i, axis=0).tolist()  # Convert to list for better readability
        max_values_list.append(max_values)
        min_values_list.append(min_values)
    else:
        max_values_list.append([0, 0, 0])  # Default values if no data
        min_values_list.append([0, 0, 0])  # Default values if no data

print("Max values for each color category:", max_values_list)
print("Min values for each color category:", min_values_list)

HSV index set to: 0 (brown)
Max values for each color category: [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
Min values for each color category: [[0, 0, 0], [0, 0, 0], [0, 0, 0]]


In [29]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import numpy as np
def rgb_to_hsv(rgb_array):
    """
    Convert an array of RGB values to HSV values.

    Parameters:
    - rgb_array: A numpy array of RGB values with shape (N, 3), where N is the number of RGB values.

    Returns:
    - hsv_array: A numpy array of HSV values with the same shape as the input.
    """
    # Ensure the input is a numpy array
    rgb_array = np.array(rgb_array, dtype=np.uint8)

    # Convert RGB to BGR (OpenCV uses BGR by default)
    bgr_array = rgb_array[:, [2, 1, 0]]

    # Reshape the array to a 1xN image (for OpenCV compatibility)
    bgr_image = bgr_array.reshape((1, -1, 3))

    # Convert BGR to HSV
    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)

    # Reshape back to the original shape
    hsv_array = hsv_image.reshape(-1, 3)

    return hsv_array

def hsv_to_rgb(hsv_array):
    """
    Convert an array of HSV values to RGB values.

    Parameters:
    - hsv_array: A numpy array of HSV values with shape (N, 3), where N is the number of HSV values.

    Returns:
    - rgb_array: A numpy array of RGB values with the same shape as the input.
    """
    # Ensure the input is a numpy array
    hsv_array = np.array(hsv_array, dtype=np.uint8)

    # Reshape the array to a 1xN image (for OpenCV compatibility)
    hsv_image = hsv_array.reshape((1, -1, 3))

    # Convert HSV to BGR
    bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    # Reshape back to the original shape
    rgb_array = rgb_image.reshape(-1, 3)

    return rgb_array
# Assuming 'result' is your image array
plot_img = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

# Get image dimensions
height, width, _ = plot_img.shape

# Generate points every 10 pixels
step_size = 10
x_coords = np.arange(0, width, step_size)
y_coords = np.arange(0, height, step_size)
grid_x, grid_y = np.meshgrid(x_coords, y_coords)
sampled_points = np.column_stack((grid_x.ravel(), grid_y.ravel()))

# Sample the colors at the points
colors = plot_img[sampled_points[:, 1], sampled_points[:, 0]]
normalized_colors = colors / 255
print(normalized_colors)
hsv_sampled_colors = rgb_to_hsv(colors)

# Normalize HSV values to the range [0, 1] for Plotly
hsv_sampled_colors = hsv_sampled_colors.astype(float)

x = hsv_sampled_colors[:, 0]  # Hue
y = hsv_sampled_colors[:, 1]  # Saturation
z = hsv_sampled_colors[:, 2]  # Value

# Initial x value and color for the draggable plane
initial_x = 0
initial_color = '#8B4513'  # Initial color is brown

# Create the 3D scatter plot
scatter = go.Scatter3d(
    x=x, y=y, z=z, mode='markers',
    marker=dict(
        size=5,
        color=['rgb({}, {}, {})'.format(r, g, b) for r, g, b in colors]
    ),
)


# Function to create a plane with a specific color
def create_plane(x_value, color):
    return go.Surface(
        z=[[min(z), min(z)], [max(z), max(z)]],
        y=[[min(y), max(y)], [min(y), max(y)]],
        x=[[x_value, x_value], [x_value, x_value]],
        opacity=0.5,
        surfacecolor=[[0, 0], [0, 0]],  # Set a uniform surfacecolor
        colorscale=[[0, color], [1, color]],  # Use the specified color
        showscale=False  # Hide the color scale
    )


# Create the initial figure
fig = go.Figure(data=[scatter, create_plane(initial_x, initial_color)])

# Set initial layout
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[min(x), max(x)]),
        yaxis=dict(range=[min(y), max(y)]),
        zaxis=dict(range=[min(z), max(z)]),
        camera=dict(
            eye=dict(x=1.25, y=1.25, z=1.25)
        )
    )
)
thresholds = np.zeros(6, np.float32)
saturation_thresholds = np.zeros(3, np.float32)
window_up = False
# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div(style={'height': '100vh', 'margin': '0', 'display': 'flex', 'flexDirection': 'column'}, children=[
    dcc.Graph(
        id='3d-plot',
        figure=fig,
        style={'height': '70vh', 'width': '100%'}
    ),
    html.Div([
        html.Div([
            html.Label('X-Axis Position:', style={'fontSize': '18px', 'marginRight': '10px'}),
            dcc.Slider(
                id='x-slider',
                min=min(x),
                max=max(x),
                step=1,
                value=initial_x,
                marks={i: {'label': str(i), 'style': {'fontSize': '14px'}} for i in
                       range(int(min(x)), int(max(x)) + 1)},
                tooltip={'placement': 'bottom', 'always_visible': True},
                updatemode='drag'
            ),
        ], style={'width': '80%', 'margin': '20px auto'}),
        html.Div([
            html.Button('Save Slider Value', id='save-button', n_clicks=0,
                        style={'fontSize': '16px', 'padding': '10px', 'marginRight': '10px'}),
            html.Div(id='saved-value', style={'fontSize': '16px', 'marginLeft': '10px'}),
        ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'margin': '10px 0'}),
        html.Div([
            html.Button('Light Brown', id='light-brown-button', n_clicks=0,
                        style={'fontSize': '14px', 'padding': '8px', 'margin': '5px'}),
            html.Button('Dark Brown', id='dark-brown-button', n_clicks=0,
                        style={'fontSize': '14px', 'padding': '8px', 'margin': '5px'}),
            html.Button('Light Blue', id='light-blue-button', n_clicks=0,
                        style={'fontSize': '14px', 'padding': '8px', 'margin': '5px'}),
            html.Button('Dark Blue', id='dark-blue-button', n_clicks=0,
                        style={'fontSize': '14px', 'padding': '8px', 'margin': '5px'}),
            html.Button('Light Red', id='light-red-button', n_clicks=0,
                        style={'fontSize': '14px', 'padding': '8px', 'margin': '5px'}),
            html.Button('Dark Red', id='dark-red-button', n_clicks=0,
                        style={'fontSize': '14px', 'padding': '8px', 'margin': '5px'}),
        ], style={'display': 'flex', 'justifyContent': 'center', 'flexWrap': 'wrap'}),
    ], style={'flex': '1', 'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'center',
              'alignItems': 'center', 'backgroundColor': '#f0f0f0', 'padding': '20px'}),
    dcc.Store(id='plane-color', data=initial_color),  # Store for the plane color
    dcc.Store(id='thresholds', data=np.zeros(6).tolist())  # Store for thresholds array
])


@app.callback(
    Output('3d-plot', 'figure'),
    Input('x-slider', 'value'),
    Input('3d-plot', 'relayoutData'),
    State('plane-color', 'data')
)
def update_figure(x_value, relayout_data, plane_color):
    # Create a new plane with the updated position and color
    new_plane = create_plane(x_value, plane_color)

    # Create the updated figure with the new plane
    updated_fig = go.Figure(data=[scatter, new_plane])

    # Preserve the camera position if available
    if relayout_data and 'scene.camera' in relayout_data:
        camera = relayout_data['scene.camera']
    else:
        camera = fig.layout.scene.camera

    # Update layout with preserved camera position
    updated_fig.update_layout(
        scene=dict(
            xaxis=dict(range=[min(x), max(x)]),
            yaxis=dict(range=[min(y), max(y)]),
            zaxis=dict(range=[min(z), max(z)]),
            camera=camera
        )
    )

    return updated_fig


@app.callback(
    Output('plane-color', 'data'),
    Input('light-brown-button', 'n_clicks'),
    Input('dark-brown-button', 'n_clicks'),
    Input('light-blue-button', 'n_clicks'),
    Input('dark-blue-button', 'n_clicks'),
    Input('light-red-button', 'n_clicks'),
    Input('dark-red-button', 'n_clicks'),
    prevent_initial_call=True
)
def update_color(light_brown, dark_brown, light_blue, dark_blue, light_red, dark_red):
    ctx = dash.callback_context

    if not ctx.triggered:
        return initial_color

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    preset_colors = {
        'light-brown-button': '#D2B48C',  # Light brown
        'dark-brown-button': '#5C4033',  # Dark brown
        'light-blue-button': '#ADD8E6',  # Light blue
        'dark-blue-button': '#00008B',  # Dark blue
        'light-red-button': '#FFA07A',  # Light red
        'dark-red-button': '#8B0000'  # Dark red
    }

    return preset_colors.get(button_id, initial_color)


@app.callback(
    Output('saved-value', 'children'),
    Input('save-button', 'n_clicks'),
    State('x-slider', 'value'),
    State('plane-color', 'data')
)
def save_slider_value(n_clicks, slider_value, plane_color):
    global thresholds  # Declare the use of the global variable
    if n_clicks > 0:
        # Determine the index based on the color
        color_indices = {
            '#D2B48C': 0,  # Light brown
            '#5C4033': 1,  # Dark brown
            '#ADD8E6': 2,  # Light blue
            '#00008B': 3,  # Dark blue
            '#FFA07A': 4,  # Light red
            '#8B0000': 5  # Dark red
        }

        # Get the index for the current plane color
        index = color_indices.get(plane_color, -1)

        if index != -1:
            # Update the thresholds array
            thresholds[index] = slider_value

            # Save the updated array back to the store
            return f'Saved Slider Value: {slider_value} at index {index}'

        return 'Invalid color selected.'

    return 'Click the button to save the slider value.'


if __name__ == '__main__':
    app.run_server(debug=True)


[[0.81960784 0.81960784 0.84705882]
 [0.89411765 0.86666667 0.8627451 ]
 [0.88235294 0.87058824 0.87843137]
 ...
 [0.96862745 0.96078431 0.96862745]
 [0.9372549  0.92156863 0.94901961]
 [0.84705882 0.82745098 0.83921569]]


In [18]:
print(thresholds)

[  0.  22. 103. 117.   0. 179.]


In [23]:
if result is None:
    print("Error: Image not loaded")
    exit()

# Convert thresholds to appropriate dtype
live_low_brown = np.array([thresholds[0], 50, 40], dtype=np.uint8)
live_high_brown = np.array([thresholds[1], 100, 240], dtype=np.uint8)
live_low_blue = np.array([thresholds[2], 70, 50], dtype=np.uint8)
live_high_blue = np.array([thresholds[3], 255, 255], dtype=np.uint8)
live_low_red = np.array([thresholds[4], 50, 50], dtype=np.uint8)
live_high_red = np.array([thresholds[5], 255, 255], dtype=np.uint8)

live_hsv = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)

# Create masks based on the color ranges
live_blue_mask = cv2.inRange(live_hsv, live_low_blue, live_high_blue)
live_red_mask = cv2.inRange(live_hsv, live_low_red, live_high_red)
live_brown_mask = cv2.inRange(live_hsv, live_low_brown, live_high_brown)

# Check masks
print("Blue mask unique values:", np.unique(live_blue_mask))
print("Red mask unique values:", np.unique(live_red_mask))
print("Brown mask unique values:", np.unique(live_brown_mask))

# Create a blank image with the same dimensions as the original image
live_overlay = np.zeros_like(result)

# Assign colors to the overlay based on masks
live_overlay[live_blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
live_overlay[live_red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
live_overlay[live_brown_mask != 0] = [0, 255, 0]  # Brown mask as green (BGR format)

# Combine the overlay with the original image
live_mask = cv2.addWeighted(result, 0.7, live_overlay, 0.3, 0)

# Display the result
cv2.namedWindow("live_mask", cv2.WINDOW_AUTOSIZE)
cv2.imshow("live_mask", live_mask)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()

Blue mask unique values: [  0 255]
Red mask unique values: [  0 255]
Brown mask unique values: [  0 255]
